# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [6]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Sun Oct 14 20:49:40 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s


NOTES:
NOTES: TODO




In [7]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Sun Oct 14 20:49:41 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRoleBinding
NAME    AGE
seldon  1s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        1s

==> v1beta1/ClusterRole
NAME        AGE
seldon-crd  1s

==> v1beta1/Role
ambassador    1s
seldon-local  1s

==> v1beta1/RoleBinding
NAME        AGE
ambassador  1s

==> v1/RoleBinding
seldon  1s

==> v1/Service
NAME                          TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador-admin  NodePort   10.111.210.204  <none>       8877:31652/TCP                 1s
seldon-core-ambassador        NodePort   10.101.200.250  <none>       8080:32625/TCP                 1s
seldon-core-seldon-apiserver  NodePort   10.110.40.180   <none>       8080:30975/TCP,5000:32443/TCP  1s
seldon-core-redis             ClusterIP  10.105.48.94    <none>       6379/TCP                       1s

==> v1beta1/Deployment
NAME                    

In [8]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

Install gRPC modules for the prediction protos.

In [9]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

## Serve Single Model

In [10]:
!helm install ../helm-charts/seldon-single-model --name mymodel

NAME:   mymodel
LAST DEPLOYED: Sun Oct 14 20:51:00 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [11]:
!helm template ../helm-charts/seldon-single-model --name mymodel | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mymodel"
    },
    "spec": {
        "name": "mymodel",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePeriodSecond

In [12]:
!kubectl rollout status deploy/mymodel-mymodel-classifier-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch

Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-classifier-0" successfully rolled out
deployment "mymodel-mymodel-svc-orch" successfully rolled out


### Get predictions

In [13]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [14]:
r = rest_request_ambassador("mymodel",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "hsiqcvkovr2n317f535i0jeor6",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.062013911980683]
    }
  }
}


#### gRPC Request

In [15]:
grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "426s4d4ked8t7lb0efdsg3tq97"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07317253279179549
  }
}

In [16]:
!helm delete mymodel --purge

release "mymodel" deleted


## Serve AB Test

In [17]:
!helm install ../helm-charts/seldon-abtest --name myabtest

NAME:   myabtest
LAST DEPLOYED: Sun Oct 14 20:52:31 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME      AGE
myabtest  0s




In [18]:
!helm template ../helm-charts/seldon-abtest --name myabtest | pygmentize -l json

---
# Source: seldon-abtest/templates/ab_test_1pod.json


---
# Source: seldon-abtest/templates/ab_test_2pods.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "myabtest"
    },
    "spec": {
	"name": "myabtest",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1M

In [19]:
!kubectl rollout status deploy/myabtest-abtest-classifier-1-0
!kubectl rollout status deploy/myabtest-abtest-classifier-1-0
!kubectl rollout status deploy/myabtest-abtest-svc-orch 

deployment "myabtest-abtest-classifier-1-0" successfully rolled out
deployment "myabtest-abtest-classifier-1-0" successfully rolled out
Waiting for deployment "myabtest-abtest-svc-orch" rollout to finish: 0 of 1 updated replicas are available...
deployment "myabtest-abtest-svc-orch" successfully rolled out


### Get predictions

In [20]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [21]:
r = rest_request_ambassador("myabtest",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "romhq0ortlbhcosq08nle1dnj6",
    "tags": {
    },
    "routing": {
      "myabtest": 1
    },
    "requestPath": {
      "myabtest": "",
      "classifier-2": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.06798732540491603]
    }
  }
}


#### gRPC Request

In [22]:
grpc_request_ambassador("myabtest",API_AMBASSADOR)

meta {
  puid: "ugv38o8aq1pkl38nvf0m1tv4c8"
  routing {
    key: "myabtest"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07245218440685676
  }
}

In [23]:
!helm delete myabtest --purge

release "myabtest" deleted


## Serve Multi-Armed Bandit

In [24]:
!helm install ../helm-charts/seldon-mab --name mymab

NAME:   mymab
LAST DEPLOYED: Sun Oct 14 20:52:58 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME   AGE
mymab  0s




In [25]:
!helm template ../helm-charts/seldon-mab --name mymab | pygmentize -l json

---
# Source: seldon-mab/templates/mab.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "mymab"
    },
    "spec": {
	"name": "mymab",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }
			],
			"terminationGracePeriodSeconds": 20
			}


In [26]:
!kubectl rollout status deploy/mymab-abtest-classifier-1-0
!kubectl rollout status deploy/mymab-abtest-classifier-2-1
!kubectl rollout status deploy/mymab-abtest-eg-router-2 

Waiting for deployment "mymab-abtest-classifier-1-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-abtest-classifier-1-0" successfully rolled out
Waiting for deployment "mymab-abtest-classifier-2-1" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-abtest-classifier-2-1" successfully rolled out
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed

### Get predictions

In [27]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [28]:
r = rest_request_ambassador("mymab",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "l6nll08e5msr8s39cpumsptokr",
    "tags": {
    },
    "routing": {
      "eg-router": 0
    },
    "requestPath": {
      "eg-router": "seldonio/mab_epsilon_greedy:1.1",
      "classifier-1": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08857813159319305]
    }
  }
}


#### gRPC Request

In [29]:
grpc_request_ambassador("mymab",API_AMBASSADOR)

meta {
  puid: "ompco5g59k1jj2svr1db7h3djm"
  routing {
    key: "eg-router"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08072007852083082
  }
}

In [30]:
!helm delete mymab --purge

release "mymab" deleted


## Serve Single Model with Outlier Detector

In [31]:
!helm install ../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true 

NAME:   mymodel
LAST DEPLOYED: Sun Oct 14 20:53:57 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [32]:
!helm template ../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mymodel"
    },
    "spec": {
        "name": "mymodel",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePeriodSecond

In [33]:
!kubectl rollout status deploy/mymodel-mymodel-classifier-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch
!kubectl rollout status deploy/mymodel-mymodel-outlier-detector-1

Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-classifier-0" successfully rolled out
Waiting for deployment "mymodel-mymodel-svc-orch" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-svc-orch" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-svc-orch" successfully rolled out
Waiting for deployment "mymodel-mymodel-outlier-detector-1" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-outlier-detector-1" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec updat

### Get predictions

In [34]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [35]:
r = rest_request_ambassador("mymodel",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "7e04p8pfklgnnimj3vc794tr6k",
    "tags": {
      "outlierScore": [0.0]
    },
    "routing": {
      "outlier-detector": -1
    },
    "requestPath": {
      "outlier-detector": "seldonio/outlier_mahalanobis:0.3",
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08955540764238194]
    }
  }
}


#### gRPC Request

In [36]:
grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "pjil5hcqmh6m7ar1rmu2db5abi"
  tags {
    key: "outlierScore"
    value {
      list_value {
        values {
          number_value: 0.0
        }
      }
    }
  }
  routing {
    key: "outlier-detector"
    value: -1
  }
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "outlier-detector"
    value: "seldonio/outlier_mahalanobis:0.3"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09630682149894522
  }
}

In [37]:
!helm delete mymodel --purge

release "mymodel" deleted
